In [1]:
import matplotlib
from EEG import *
# from CNT import prssigma,prstheta
from pptx import Presentation
from pptx.util import Inches
path_to_file = "G:/Processed_EEG/EGI"
fnames = os.listdir(path_to_file)
fnames = [f for f in fnames if len(f.split("."))==1]
paths = [path_to_file+"/"+fname+".str" for fname in fnames]
# path_to_files = getmmfaddress(root_dir)
# pathsegi = [pathh for pathh in path_to_files if isinstance(pathh, str)]
# pathsegi.remove(pathsegi[5])
# pathsegi = getpklfiles(root_dir,"Artifacts")
# pathsegi = getpklfiles(root_dir)
# pathsegi = getpklfiles(root_dir,"dtrrevstim")
# pathsegi = getpklfiles(root_dir,"dtrrevstimdist")
#No 5 is too noisy 
ages = ["32,5,F,yes",
        "32,4,M,yes",
        "32,1,M,yes",
        "32,3,M,yes",
        "32,0,M,yes",
        "32,4,F,yes",
        "32,6,M,yes",
        "31,5,M,yes"
        ]
# eeg = []
# events=[]
# delta=[]
# brush=[]
# plt.close("all")
# prs = Presentation("G:/Processed_EEG/Present/Brushband1deltabrushPLV.pptx")
# with open("G:\\datasets\\plvsTB.pkl" ,"rb") as f:
#     dataset,subnum = pickle.load(f)
# prs = Presentation("EGI.pptx") 
prs = Presentation() 
prssigma = Presentation()
prstheta = Presentation()
TC_slide = prssigma.slide_layouts[1]
blank_slide_layout = prstheta.slide_layouts[6]
# TC_slide = prs.slide_layouts[1]
# TC_slide = prs.slide_layouts[3]
# blank_slide_layout = prs.slide_layouts[6]
bands = {"brushband0":"Theta band","brushband1":"Sigma band"}
cmean = lambda x,axis:sp.stats.circmean(x,np.pi,-np.pi,axis)
handpicked_atoms_EGI = {"brushband0":{"lp":[],"rp":[],"left":[],"right":[]},
                    "brushband1":{"lp":[],"rp":[],"left":[],"right":[]}}
for i,f in tqdm(enumerate(paths)):    
    # with open(f ,"rb") as file:
        # eeg.append(pickle.load(file))
        # eeg = pickle.load(file) 
        # eeg.savedata(mode="distributed",postfix = "dtrrevstimdist",address="D:\Processed_EEG\EGI")
    # ppt_kw = {"title":f"EGI subject No {i}"} 
    # plot_kw = {"ordered":True,"bands":"brushband1"}
    # eegs.append(EEG(f,what=["selection_atom","fname","savefolder"]))
    eeg =EEG(f)   
      
    # eeg.data.interpolate_bads()
    # eeg.picks = mne.pick_types(eeg.data.info,eeg=True)
    # eeg.cleandata(thmethod = "data",Threshold=200e-6)
    # eeg.cleandata("grad","data",Threshold=8e-5)    
    # eeg.postprocesstemp(mode="artifacts")
    # eeg.set_BAD_indices()
    # eeg.average_montage()
    # eeg.savedata(mode="distributed")

    # mode = {"selection" : "by_env","unique":False}
    # eeg.sort_activations(mode=mode)    
    # eeg.calc_plv("all","fixed_window",3.5,7,12,20)
    # eeg.get_delta_spec()
    # eeg.findmcscdb()

    # annots = eeg.data.annotations.copy()
    # eeg.data = eeg.dataraw.copy()
    # eeg.data.interpolate_bads()
    # eeg.picks = mne.pick_types(eeg.data.info,eeg=True)
    # eeg.data.set_annotations(annots)
    # eeg.set_BAD_indices()
    # eeg.average_montage()
    # eeg.plot_learned_atoms()
    # eeg.calc_plv("all","fixed_window",3.5,8,12,20)
    # eeg.findmcscdb()
    # eeg.get_plv()
    # eeg.savedata(mode="distributed")
    # eeg.plot_atoms_plv()
    # eeg.data = eeg.dataraw
    # eeg.data.interpolate_bads()
    # eeg.picks = mne.pick_types(eeg.data.info,eeg=True)
    # eeg.postprocesstemp(mode="artifacts")
    # eeg.average_montage()

#% Extractng all the selected atoms
    """ 
    for band in eeg.selection_atom.keys():
        for location in eeg.selection_atom[band].keys():
            cdl = eeg.learned["multivaraite"][band][0]
            if eeg.selection_atom[band][location] is None:
                pass
            else:
                handpicked_atoms_EGI[band][location].append((cdl.u_hat_[eeg.selection_atom[band][location]],
                                                            cdl.v_hat_[eeg.selection_atom[band][location]]))
    """
#% plotting plv versus distance to stimuli and generating dataframe 
    """     
    dp = []    
    for band in eeg.mcscdetepochs["by_acts"].keys():
        for loc in eeg.selection_atom[band].keys():
            selected = eeg.selection_atom[band][loc]
            if selected is None:
                continue
            try:
                epochs = eeg.mcscdetepochs["by_acts"][band][selected][eeg.mcscdbind["by_acts"][band][selected]]
                events = epochs.events
                dist = eeg.get_distance_to_stimuli(events = events,origin=0)
                phases = cmean(eeg.coupling_phase["by_acts"][band][selected][1],axis=-1)[eeg.mcscdbind["by_acts"][band][selected]]
            except TypeError:
                continue
            dp.append((band,loc,dist,phases))
            df = df.append(pd.DataFrame({"band":[band]*len(phases),
                       "loc":[loc]*len(phases),
                       "dist":dist,
                       "device":["EGI"]*len(phases),
                       "plv":phases,
                       "duration(min)":len(eeg.cleanindex)/256/60,
                       "subject":[i+4]*len(phases)})
                      ,ignore_index=True)
    try:
        fig,plots = plt.subplots(2,int(np.ceil(len(dp)/2)),subplot_kw={"projection":"polar"})
    except:
        continue
    for axrow in plots:
        try:
            for ax in axrow:
                try:
                    toplot = dp.pop()
                    ax.scatter(toplot[-1],toplot[-2],s=10)
                    earlymean = cmean(toplot[-1][toplot[-2]<4],axis=0)
                    latemean = cmean(toplot[-1][toplot[-2]>4],axis=0)
                    ax.arrow(earlymean,0,0,15,width=0.1,label="early",color="green")
                    ax.arrow(latemean,0,0,15,width=0.1,label="late",color="yellow")
                    ax.set_title("band {},\nLocation:{}".format(toplot[0],toplot[1]))
                    ax.set_rmax(15)
                    ax.legend()
                except IndexError:
                    continue
        except Exception:
            ax=axrow            
            try:
                toplot = dp.pop()
                ax.scatter(toplot[-1],toplot[-2],s=10)
                earlymean = cmean(toplot[-1][toplot[-2]<4],axis=0)
                latemean = cmean(toplot[-1][toplot[-2]>4],axis=0)
                ax.arrow(earlymean,0,0,15,width=0.1,label="early",color="green")
                ax.arrow(latemean,0,0,15,width=0.1,label="late",color="yellow")
                ax.set_title("band {},\nLocation:{}".format(toplot[0],toplot[1]))
                ax.set_rmax(15)
                ax.legend()
            except IndexError:
                continue            
        
    slide =  prs.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "The phase of the detected events plotted versus the distance to the previous stimuli of EGI subject {}".format(i)
    fig.savefig("sth1.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth1.png",ph.left,ph.top,height=ph.height)
    plt.close(fig) 
    """
#% Generating dataframe for PLVs
    """ 
    # df = eeg.get_report_mcsc()
    # dataset = dataset.append(df)
    for method in eeg.plv.keys():        
        for band in eeg.plv[method]:
            for window in eeg.plv[method][band][0].keys():
                for location in eeg.selection_atom[band]:
                    if eeg.selection_atom[band][location] is None:
                        bb = np.nan
                    else:
                        aa = eeg.plv[method][band][eeg.selection_atom[band][location]][window]
                        bb = aa.mean()
                    name = "sub-{}".format(subnum)
                    dataset = dataset.append({"band":band,
                                    "window":window,
                                    "name":name,
                                    "fullname":eeg.fname,
                                    "location":location,
                                    "events": 0 if eeg.selection_atom[band][location] is None else len(eeg.mcscdbind["by_acts"][band][eeg.selection_atom[band][location]])
                                    ,
                                    "device":"EGI",
                                    "plv_phase":np.angle(bb),
                                    "plv_strength":np.abs(bb),
                                    "age":"high",
                                    "duration(min)":len(eeg.cleanindex)/256/60,
                                    "stimuli":"yes"}
                                   ,ignore_index=True)
    subnum+=1    
    """
#% Plotting selected atoms
    """         
    for band in eeg.selection_atom.keys():
        for pos in eeg.selection_atom[band]:
            selected = eeg.selection_atom[band][pos]
            if selected is None:
                continue
            plot_kw = {"ordered":True,"bands":band,"which":[selected],"method":"deltabrush"}
            eeg.plot_atoms_plv(plot_kw=plot_kw)
            slide =  prs.slides.add_slide(TC_slide)
            slide.placeholders[0].text = "The detected events of atom {} in {} subject No {}".format(selected,bands[band],i)
            fig = plt.gcf()
            fig.savefig("sth1.png")
            ph = slide.placeholders[1]
            slide.shapes.add_picture("sth1.png",ph.left,ph.top,height=ph.height)
            plt.close(fig)
            try:
                epdb = eeg.mcscdetepochs["by_acts"][band][selected][eeg.mcscdbind["by_acts"][band][selected]]
            except TypeError:
                pass
            if len(epdb.events)<=20:
                continue
            dataraw = eeg.data.get_data()
            dataraw[eeg.badsegments==1]=0
            dataio = mne.io.RawArray(dataraw,eeg.data.info)
            epochs = mne.Epochs(dataio,epdb.events,tmin=-2,tmax=2,baseline=None)
            ev = epochs.average()
            ev.plot_joint()
            fig = plt.gcf()
            fig.savefig("sth1.png")
            asp = fig.get_figwidth()/fig.get_figheight()
            slide.shapes.add_picture("sth1.png",ph.left+ph.width/4,ph.top,height=ph.height/2,width = ph.height/2*asp)
            plt.close(fig)
            
            epochs2 = epochs.load_data().copy().pick(eeg.coupling_channels["by_acts"][band][selected])
            ev = epochs2.average()            
            for evoked in ev.data:
                plt.plot(ev.times,evoked)
            plt.legend(eeg.coupling_channels["by_acts"][band][selected])
            nave = "Nave = {}".format(ev.nave)
            plt.text(0,evoked.max(),nave)
            fig = plt.gcf()
            fig.savefig("sth1.png")            
            asp = fig.get_figwidth()/fig.get_figheight()
            slide.shapes.add_picture("sth1.png",ph.left+ph.width/4,ph.top+ph.height/2,height=ph.height/2,width = ph.height/2*asp)
            plt.close(fig)
            
            # hmany = len(epochs)
            # colors = np.array(["black"]*len(epochs.ch_names))            
            # choi = mne.pick_channels(epochs.ch_names,eeg.coupling_channels["by_acts"][band][selected])
            # colors[choi] = "blue"
            # colors = [colors.tolist()]*8
            # for a in range(0,hmany,8):
            #     epplot = epochs[a:a+8]
            #     fig = epplot.plot(n_channels=63,epoch_colors = colors[:len(epplot)])
            #     fig.savefig("sth1.png")
            #     slide = prs.slides.add_slide(blank_slide_layout)
            #     left = top = 0
            #     PIC = slide.shapes.add_picture('sth1.png', left-0.1*prs.slide_width, top, height = prs.slide_height)
            #     plt.close(fig)       
    """

#% Plotting selected atoms with their topomap (Old version)    
    """ 
    for band in eeg.selection_atom.keys():
        for pos in eeg.selection_atom[band]:
            selected = eeg.selection_atom[band][pos]
            if selected is None:
                continue
            plot_kw = {"ordered":True,"bands":band,"which":[selected]}
            eeg.plot_atoms_plv(plot_kw=plot_kw)
            epcs = eeg.mcscdetepochs["by_acts"][band]
            epcsb11 = epcs[selected]
            dptp = np.array(eeg.deltaptp["by_acts"][band][selected]).mean(axis=1)
            coupstr = eeg.coupling_strength["by_acts"][band][selected][1].mean(axis=1)
            sactsb11 = eeg.sorted_acts["by_acts"][band][selected]
            ldptp = np.log(dptp)
            pos = np.percentile(ldptp,50)
            dptps = sigmoid(ldptp,0.3,pos)
            # dptps = minmax_scale(dptp.reshape(-1,1))
            coupstrs = sigmoid(coupstr,0.1,0.5)
            # coupstrs = minmax_scale(coupstr.reshape(-1,1))
            lsactsb11 = np.log(sactsb11)
            pos = np.percentile(lsactsb11,50)
            sactss = sigmoid(lsactsb11,0.3,pos)
            # sactss = minmax_scale(sactsb11.reshape(-1,1))
            # allmeasure = np.concatenate((dptps,coupstrs,sactss),axis=1)
            allmeasure = np.c_[dptps,coupstrs,sactss]
            sorter = allmeasure.prod(axis=1)
            data2 = eeg.data.copy()
            ann = mne.annotations.Annotations([],[],[])
            data2.set_annotations(ann)
            allep = mne.Epochs(data2,epcsb11.events,tmin=-2,tmax=2,baseline=None,preload=True)
            colors = np.array(["black"]*len(allep.ch_names))
            
            choi = mne.pick_channels(allep.ch_names,eeg.coupling_channels["by_acts"][band][selected])
            colors[choi] = "blue"
            colors = [colors.tolist()]*8
            sorter = sorter[allep.selection]
            slide =  prs.slides.add_slide(TC_slide)
            slide.placeholders[0].text = "The detected events of atom {} in {} subject No {}".format(selected,band,i)
            fig = gcf()
            fig.savefig("sth2.png")
            ph = slide.placeholders[1]
            slide.shapes.add_picture("sth2.png",ph.left,ph.top,height=ph.height)
            plt.close(fig)
            ss = sorter[sorter.argsort()[::-1]]
            hmany = np.arange(len(ss))[ss<0.5][0]
            # colors = [colors.tolist()]*hmany
            title = "eeg number {} band: {} atom:{} events:{}".format(i,band,selected,hmany)
            ev = allep[sorter.argsort()[::-1]][:hmany].average(picks = choi)
            for evoked in ev.data:
                plt.plot(evoked)
            plt.legend(eeg.coupling_channels["by_acts"][band][selected])
            nave = "Nave = {}".format(hmany)
            plt.text(0,evoked.max(),nave)
            fig = gcf()
            fig.savefig("sth2.png")
            ph = slide.placeholders[2]
            slide.shapes.add_picture("sth2.png",ph.left-1,ph.top)
            plt.close(fig)
            break
            for a in range(0,hmany,8):
                fig = allep[sorter.argsort()[::-1]][a:a+8].plot(n_channels=63,epoch_colors = colors)
                fig.savefig("sth2.png")
                slide = prs.slides.add_slide(blank_slide_layout)
                left = top = 0
                PIC = slide.shapes.add_picture('sth2.png', left-0.1*prs.slide_width, top, height = prs.slide_height)
                plt.close(fig)
    """
        
        
    # eeg.savefolder = "G:\Processed_EEG\EGI"
#% Plotting all the learned atoms   
    
    plot_kw = {"ordered":True,"bands":"brushband0","method":"deltabrush",
               "which":np.arange(5)}
    eeg.plot_atoms_plv(plot_kw=plot_kw)
    slide =  prstheta.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "Theta band, atoms 0 to 4 EGI sub {}".format(i)
    fig = plt.gcf()
    fig.savefig("sth.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    plt.close(fig)
    
    # plot_kw = {"ordered":True,"bands":"brushband0","method":"deltabrush",
    #            "which":np.arange(5,10)}
    # eeg.plot_atoms_plv(plot_kw=plot_kw)
    # slide =  prstheta.slides.add_slide(TC_slide)
    # slide.placeholders[0].text = "Theta band, atoms 5 to 9 EGI sub {}".format(i)
    # fig = plt.gcf()
    # fig.savefig("sth.png")
    # ph = slide.placeholders[1]
    # slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    # plt.close(fig) 

    plot_kw = {"ordered":True,"bands":"brushband1","method":"deltabrush",
               "which":np.arange(5)}
    eeg.plot_atoms_plv(plot_kw=plot_kw)
    slide =  prssigma.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "Sigma band, atoms 0 to 4 EGI sub {}".format(i)
    fig = plt.gcf()
    fig.savefig("sth.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    plt.close(fig)
    
    # plot_kw = {"ordered":True,"bands":"brushband1","method":"deltabrush",
    #            "which":np.arange(5,10)}
    # eeg.plot_atoms_plv(plot_kw=plot_kw)
    # slide =  prssigma.slides.add_slide(TC_slide)
    # slide.placeholders[0].text = "Sigma band, atoms 5 to 9 EGI sub {}".format(i)
    # fig = plt.gcf()
    # fig.savefig("sth.png")
    # ph = slide.placeholders[1]
    # slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    # plt.close(fig)    
   
#% TO be defined later 
    """ phases.append(eeg.mean_phase["brushband0"])
    selection_atom["brushband0"]=selection[i][:3]
    selection_atom["brushband1"]=selection[i][3:]
    eeg.selection_atom=selection_atom
    eeg.selection_detection = eeg.selection_atom 
    eeg.calc_atom_detection()
    eeg.plot_atoms_plv(ppt_kw=ppt_kw,plot_kw=plot_kw)
    plt.close("all")    
    eeg.plot_atoms_plv(ppt_kw=ppt_kw)
    plt.close("all")
    eeg.calc_plv()    
    eeg.convertres()
    eeg.savedata(mode="annotations")
    annotadd = "_".join([f.split(".")[0],"annotations.pkl"])
    with open(annotadd,"rb") as f:
        data = pickle.load(f)
    eeg.annotate_events(data,mode="reload")
    eeg.Convert()
    eeg.set_BAD_indices()
    eeg.average_montage()  """
    
    #% Calculate sparse atoms
    """ 
    chnames = np.array(eeg.data.info["ch_names"])[eeg.picks].tolist()
    eeg.chnames = chnames    
    print("___________________________________________{}th data___________________________________".format(i))    
    gcdl = {"n_atoms":5,
        "n_times_atom": 1, # in second and will be converted to samples 
        "rank1":True,
        "uv_constraint":"separate",
        "window":True,
        "unbiased_z_hat":True,
        "D_init":"chunk",
        "lmbd_max":"scaled",
        "reg" : 0.1,
        "n_iter":100,
        "eps":1e-4,
        "solver_z":"lgcd",
        "solver_z_kwargs":{'tol': 1e-3,
                          'max_iter': 100000},
        "solver_d":'alternate_adaptive',
        "solver_d_kwargs":{'max_iter': 300},
        "sort_atoms":True,
        "verbose":1,
        "random_state":0,
        "n_jobs":20}
    eeg.learn_sparse(mode="multivaraite",cdl=gcdl,bands=np.array([3.5,7,12,20]))
    eeg.sort_activations()    
    eeg.calc_plv("all","fixed_window",3.5,7,12,20)
    eeg.get_delta_spec()
    eeg.findmcscdb()
    eeg.savedata(mode="distributed") 
    """
   
#% Preprocessing steps    
    
    """ 
    eeg.overlay_detection()
    eeg.get_plv()
    eeg.savedata(mode="distributed")    
    eeg.extract(which='stimuli_clean',method="perch",tmin=-6,tmax=6)
    eeg.savedata(mode="distributed",postfix = "dtrrevstimdist",address="G:\Processed_EEG\EGI")
    eeg.savedata(mode="distributed")
    print("_______________________________________________________________________________________")
    assert eeg2==eeg
    with open("/".join(f.split("\\")[:-1]) +"/RAW_data_annotations.pkl","rb") as file2:
        data = pickle.load(file2)               
        eeg.sync()                    
        eeg.set_exo_eves()
        eeg.data.plot_sensors(show_names=True)
        events.append(eeg.prepare_cluster_data())
        brush.append(eeg.prepare_cluster_data(mode="brush"))
        delta.append(eeg.prepare_cluster_data(mode="delta"))
        delpow = eeg.cluster()
        eeg.export()
        eeg.get_epoched_detection(Threshold=0.2)
        eeg.get_epoched_detection(mode = "dbman",Threshold=0.2)
        eeg.get_epoched_detection(mode="dbman_avg",Threshold=0.2)        
        eeg.pop_manual_events(name="dbman_avg")
        
        eeg.savedata(mode="annotations")
        print(eeg.history)
        eeg.set_BAD_indices()
        eeg.average_montage()
        eeg.postprocesstemp(mode="artifacts")
        eeg.find_events(mode="delta",maxduration=2.5,percentile=[80,50,70])
        eeg.find_events(method='rms') 
        eeg.postprocesstemp(mode="delta",maxduration=2.5,percentile=[80,50,70])
        eeg.postprocesstemp(mode="brush",threshold=95,minduration=0.4)
        eeg.detect_db()
        eeg.convert_annots()
        eeg.savedata(postfix="dtr")
        eeg.debug.pop("grads")
        eeg.cleandata(thmethod = "data",Threshold=200e-6)
        eeg.cleandata("grad","data",Threshold=8e-5) """

# prs.save("EGI.pptx")
# os.system('shutdown -f -s -t 0')     
# prs.save("G:/Processed_EEG/Present/EGIplvtostim1.pptx")
# prs.save("G:/Processed_EEG/Present/Brushband1deltabrushPLV.pptx")

# prs.save("G:/Processed_EEG/Presenttheta_brush/EGIplvtostim.pptx")
# prs.save("G:/Processed_EEG/Presenttheta_brush/EGISamples.pptx")
# prs.save("G:/Processed_EEG/Presenttheta_brush/EGISelectedHighN.pptx")
# prs.save("G:/Processed_EEG/Presenttheta_brush/EGISelected.pptx")

# with open("G:\\datasets\\plvs.pkl" ,"wb") as f:
#     pickle.dump([dataset,subnum],f)
# with open("G:\\datasets\\plvsTB.pkl" ,"wb") as f:
#     pickle.dump([dataset,subnum],f)
# with open("G:\datasets\plvtostim.pkl","wb") as f:
#     pickle.dump(df , f)
# with open("G:\datasets\plvtostimTB.pkl","wb") as f:
#     pickle.dump(df , f)

0it [00:00, ?it/s]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


1it [00:46, 46.36s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


2it [01:33, 46.80s/it]

...

6it [04:52, 49.63s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


7it [05:38, 48.53s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


8it [06:27, 48.45s/it]


In [2]:
# from CNT import prstheta, prssigma
path_to_file = "G:\\Processed_EEG\\EGI_No_stimuli"
fnames = os.listdir(path_to_file)
fnames = [f for f in fnames if len(f.split("."))==1]
paths = [path_to_file+"/"+fname+".str" for fname in fnames]


TC_slide = prstheta.slide_layouts[1]
blank_slide_layout = prs.slide_layouts[6]
for i,f in tqdm(enumerate(paths)):    
    eeg = EEG(f)           
    plot_kw = {"ordered":True,"bands":"brushband0","method":"deltabrush",
               "which":np.arange(5)}
    eeg.plot_atoms_plv(plot_kw=plot_kw)
    slide =  prstheta.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "Theta band, atoms 0 to 4 EGI_NO_STIM sub {}".format(i)
    fig = plt.gcf()
    fig.savefig("sth.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    plt.close(fig)
    
    # plot_kw = {"ordered":True,"bands":"brushband0","method":"deltabrush",
    #            "which":np.arange(5,10)}
    # eeg.plot_atoms_plv(plot_kw=plot_kw)
    # slide =  prstheta.slides.add_slide(TC_slide)
    # slide.placeholders[0].text = "Theta band, atoms 5 to 9 EGI_NO_STIM sub {}".format(i)
    # fig = plt.gcf()
    # fig.savefig("sth.png")
    # ph = slide.placeholders[1]
    # slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    # plt.close(fig) 

    plot_kw = {"ordered":True,"bands":"brushband1","method":"deltabrush",
               "which":np.arange(5)}
    eeg.plot_atoms_plv(plot_kw=plot_kw)
    slide =  prssigma.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "Sigma band, atoms 0 to 4 EGI_NO_STIM sub {}".format(i)
    fig = plt.gcf()
    fig.savefig("sth.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    plt.close(fig)
    
    # plot_kw = {"ordered":True,"bands":"brushband1","method":"deltabrush",
    #            "which":np.arange(5,10)}
    # eeg.plot_atoms_plv(plot_kw=plot_kw)
    # slide =  prssigma.slides.add_slide(TC_slide)
    # slide.placeholders[0].text = "Sigma band, atoms 5 to 9 EGI_NO_STIM sub {}".format(i)
    # fig = plt.gcf()
    # ph = slide.placeholders[1]
    # fig.savefig("sth.png")
    # slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    # plt.close(fig) 
# prs.save("G:/Processed_EEG/Presenttheta_brush/Brushband1_deltabrushPLV.pptx")

0it [00:00, ?it/s]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


1it [00:28, 28.88s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


2it [01:05, 33.52s/it]

...

3it [02:30, 56.98s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


4it [03:24, 55.66s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


5it [04:11, 50.20s/it]


In [3]:
# from CNT import prssigma,prstheta
path_to_file = "G:\\Processed_EEG\\EGI_30_32"
fnames = os.listdir(path_to_file)
fnames = [f for f in fnames if len(f.split("."))==1]
paths = [path_to_file+"/"+fname+".str" for fname in fnames]

# prs = Presentation("G:/Processed_EEG/Presenttheta_brush/Brushband1_deltabrushPLV.pptx") 
TC_slide = prstheta.slide_layouts[1]
blank_slide_layout = prs.slide_layouts[6]
for i,f in tqdm(enumerate(paths)):    
    eeg = EEG(f)           
    plot_kw = {"ordered":True,"bands":"brushband0","method":"deltabrush",
               "which":np.arange(5)}
    eeg.plot_atoms_plv(plot_kw=plot_kw)
    slide =  prstheta.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "Theta band, atoms 0 to 4 EGI_30_32 sub {}".format(i)
    fig = plt.gcf()
    fig.savefig("sth.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    plt.close(fig)
    
    # plot_kw = {"ordered":True,"bands":"brushband0","method":"deltabrush",
    #            "which":np.arange(5,10)}
    # eeg.plot_atoms_plv(plot_kw=plot_kw)
    # slide =  prstheta.slides.add_slide(TC_slide)
    # slide.placeholders[0].text = "Theta band, atoms 5 to 9 EGI_30_32 sub {}".format(i)
    # fig = plt.gcf()
    # fig.savefig("sth.png")
    # ph = slide.placeholders[1]
    # slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    # plt.close(fig) 

    plot_kw = {"ordered":True,"bands":"brushband1","method":"deltabrush",
               "which":np.arange(5)}
    eeg.plot_atoms_plv(plot_kw=plot_kw)
    slide =  prssigma.slides.add_slide(TC_slide)
    slide.placeholders[0].text = "Sigma band, atoms 0 to 4 EGI_30_32 sub {}".format(i)
    fig = plt.gcf()
    fig.savefig("sth.png")
    ph = slide.placeholders[1]
    slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    plt.close(fig)
    
    # plot_kw = {"ordered":True,"bands":"brushband1","method":"deltabrush",
    #            "which":np.arange(5,10)}
    # eeg.plot_atoms_plv(plot_kw=plot_kw)
    # slide =  prssigma.slides.add_slide(TC_slide)
    # slide.placeholders[0].text = "Sigma band, atoms 5 to 9 EGI_30_32 sub {}".format(i)
    # fig = plt.gcf()
    # ph = slide.placeholders[1]
    # fig.savefig("sth.png")
    # slide.shapes.add_picture("sth.png",ph.left,ph.top,height=ph.height)
    # plt.close(fig) 
prssigma.save("G:/Processed_EEG/Presenttheta_brush/Brushband1_deltabrushPLV.pptx")
prstheta.save("G:/Processed_EEG/Presenttheta_brush/Brushband0_deltabrushPLV.pptx")

0it [00:00, ?it/s]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


1it [00:42, 43.00s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


2it [01:24, 42.03s/it]

...

10it [06:24, 38.31s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


11it [06:41, 31.54s/it]

Displayed 5 atomstom                      
Displayed 5 atomstom                      


12it [07:18, 36.55s/it]
c:\Users\Mehdi\Anaconda3\lib\zipfile.py:1514: UserWarning: Duplicate name: 'ppt/slideLayouts/slideLayout2.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\Mehdi\Anaconda3\lib\zipfile.py:1514: UserWarning: Duplicate name: 'ppt/slideLayouts/_rels/slideLayout2.xml.rels'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\Mehdi\Anaconda3\lib\zipfile.py:1514: UserWarning: Duplicate name: 'ppt/slideMasters/slideMaster1.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\Mehdi\Anaconda3\lib\zipfile.py:1514: UserWarning: Duplicate name: 'ppt/slideMasters/_rels/slideMaster1.xml.rels'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\Mehdi\Anaconda3\lib\zipfile.py:1514: UserWarning: Duplicate name: 'ppt/slideLayouts/slideLayout1.xml'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\Mehdi\Anaconda3\lib\zipfile.py:1514: UserWarning: Duplicate name: 'ppt/slideLayouts/_